<h1 class="header"><img class="logo" src="../images/socib_logo.png" width="200px"></h1>

<h3> STYLING NOTEBOOK (OPTIONAL)</h3>

In [1]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('https://raw.githubusercontent.com/socib/API_examples/master/style/custom/custom.css').read())

<h3> HANDY PYTHON PACKAGES </h3>

In [2]:
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
from json2html import *
from IPython.display import HTML, display
from datetime import datetime
%matplotlib inline

<h3> PREREQUISITES </h3>

In [3]:
api_key = '' #write you SOCIB api_key
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

<h3> REQUESTING A DATA-PRODUCTS'S DATA </h3>

Let's look for data-products involving a certain platform type:

In [4]:
end_point = '/data-products/'
platform_type = 'Glider'
url_addons = "?platform_type=%s"%(platform_type)
request = requests.get('%s%s%s' % (api_url, end_point,url_addons), headers=headers)
response = json.loads(request.text)

print('Requested url: '+'%s%s%s' % (api_url, end_point,url_addons))
print(" ")
print('%s data-products contains deployments involving %s '% (response['count'], platform_type))

Requested url: http://api.socib.es/data-products/?platform_type=Glider
 
56 data-products contains deployments involving Glider 


As the pagination is 8, we will have to loop all pages to get the full list of the data-products of type glider:

In [5]:
results = response['results']
page = response['next']
while page != None:
    request = requests.get(page, headers=headers)
    response = json.loads(request.text)
    for glider in response['results']:
        results.append(glider)
    page = response['next']

In [6]:
HTML(json2html.convert(json=results))

info,description,id,name
None,Butterfly cruise serving as Engineering Trial for Slocum scientific mission,bfly-precartex,bfly preCARTEX
None,"Canales cruise, defined and followed by SOCIB (with in kind collaboration of IMEDEA-CSIC) to implement the inter-annual monitoring endurance line (ENL1) covering the Balearic channels of Mallorca and Eïvissa during the year 2013 as defined in the SOCIB's Implementation Plan 2013. The list of waypoints conforming this cruise stand as the default set for all the canales missions for the campaign of 2013; nevertheless, slightly modifications may be applied under special circumstances.",canales2013,canales2013
None,All missions with the main objective of testing all aspects of operational gliders,engineering-tests2018,engineering tests2018
None,"Galtika cruise, carried out by IMEDEA and SOCIB in the frame of the MyOcean2 project, comprises the coordinated deployment of two surface drifters and a glider in a region close to the track 16 of the SARAL satellite mission, located in the Western Mediterranean close to Eivissa island during August 2013, to allow intercalibration of altimetry data from the AltiKa altimeter, also taking advantage of data from models and the HF-radar stations available in the region.",galtika,Galtika
None,"GARICAST stands for Glider Against Rosette Inter-comparison CAST. This is the name given, at SOCIB, to the action of attaching an AUV-GLIDER to SOCIB-RV's CTD-Rosette and deploying this whole setup in order to gather CTD, OXY and FLNTU synoptic data from both observing platforms. The main goal of this effort is to validate Glider sensors taking Rosette sensors as benchmark. Consequently, GARICAST missions always occur in the frame of a SOCIB-RV cruise. GARICAST_#### stands for the annual (year ####) campaign of Glider sensor validation by means of using SOCIB-RV as calibration facility.",garicast-2017,GARICAST 2017
None,SOCIB Glider Facility Sea-Trial Tests during year 2015,gf-tests-2015,GF TESTS 2015
None,"Canales cruise, defined and followed by SOCIB (with in kind collaboration of IMEDEA-CSIC) to implement the inter-annual monitoring endurance line (ENL1) covering the Balearic channels of Mallorca and Eïvissa during the year 2014 as defined in the SOCIB's Implementation Plan 2013. The list of waypoints conforming this cruise stand as the default set for all the canales missions for the campaign of 2014; nevertheless, slightly modifications may be applied under special circumstances.",glider-canales2014,glider canales2014
None,"Canales cruise, defined and followed by SOCIB (with in kind collaboration of IMEDEA-CSIC) to implement the inter-annual monitoring endurance line (ENL1) covering the Balearic channels of Mallorca and Eïvissa during the year 2015 as defined in the SOCIB's Implementation Plan 2013. The list of waypoints conforming this cruise stand as the default set for all the canales missions for the campaign of 2015; nevertheless, slightly modifications may be applied under special circumstances.",glider-canales2015,glider canales2015
None,"Canales cruise, defined and followed by SOCIB (with in kind collaboration of IMEDEA-CSIC) to implement the inter-annual monitoring endurance line (ENL1) covering the Balearic channels of Mallorca and Eïvissa during the year 2016 as defined in the SOCIB's Implementation Plan 2013. The list of waypoints conforming this cruise stand as the default set for all the canales missions for the campaign of 2016; nevertheless, slightly modifications may be applied under special circumstances.",glider-canales2016,glider canales2016
None,"Canales cruise, defined and followed by SOCIB (with in kind collaboration of IMEDEA-CSIC) to implement the inter-annual monitoring endurance line (ENL1) covering the Balearic channels of Mallorca and Eïvissa during the year 2017. The list of waypoints conforming this cruise stand as the default set for all the canales missions for the campaign of 2017; nevertheless, slightly modifications may be ap

Next we will see how to get the data comming from one of the above data-products.

<h3>GETTING A DATA-PRODUCT'S <span>netCDFs</span></h3>

Let's focuse on the first data-platform from above list:

In [7]:
targeted_data_product = results[0] #data platform number 0

In [8]:
end_point_plus_aux = '/data-products/'+targeted_data_product['id']
request = requests.get('%s%s' % (api_url, end_point_plus_aux), headers=headers)
response = json.loads(request.text)
print('Requested url: '+'%s%s. Be aware of the name requested' % (api_url, end_point_plus_aux))

Requested url: http://api.socib.es/data-products/bfly-precartex. Be aware of the name requested


In [9]:
files_metadata = {k:v for k, v in response.items() if k != 'sources'} #skipping the product's deployment's data for later
for k, v in response.items():
    if k == 'sources':
        files_metadata[k] = len(v) 
HTML(json2html.convert(json=files_metadata))

info,None
sources,4
description,Butterfly cruise serving as Engineering Trial for Slocum scientific mission
name,bfly preCARTEX
id,bfly-precartex


In order to access the netCDFS you should request each of the composing deployment 'entries' (see [what is a data source](../data_sources/what_is_a_data_source.ipynb)):

So for example, let's loop over every netCDF available for every of the above source:

In [10]:
netCDFs = []
for source in response['sources']:
    for netCDF in source['entries']:
        netCDFs.append(netCDF)
print('This product contains a total of %s netCDFs' % (len(netCDFs)))

This product contains a total of 11 netCDFs


In [11]:
HTML(json2html.convert(json=netCDFs))

Focusing just on the services to download, explore or view:

In [12]:
netCDFs = []
for source in response['sources']:
    for netCDF in source['entries']:
        netCDFs.append(netCDF['services'])
print('This product contains a total of %s netCDFs' % (len(netCDFs)))

This product contains a total of 11 netCDFs


In [25]:
HTML(json2html.convert(json=netCDFs))